In [ ]:
import voos
import database
import pandas as pd
from tqdm import tqdm

urls = list( voos.voos_url )[ :5 ]


def download_voos( url ):
	return pd.read_csv( url, skiprows = 1, sep = ";" )


voos = pd.DataFrame()
for url in tqdm( urls ):
	parcial = download_voos( url )

	voos = pd.concat( (parcial, voos), axis = 0 )


In [ ]:
for data in [ "Chegada Prevista", "Chegada Real", "Partida Prevista", "Partida Real" ]:
	voos[ data ] = pd.to_datetime( voos[ data ], infer_datetime_format = True )

In [ ]:
voos[ "ano" ] = voos[ "Partida Real" ].dt.year
voos[ "mes" ] = voos[ "Partida Real" ].dt.month
voos[ "semana_ano" ] = voos[ "Partida Real" ].dt.week
voos[ "dia_semana" ] = voos[ "Partida Real" ].dt.dayofweek

voos[ "atraso_chegada" ] = (voos[ "Chegada Real" ] - voos[ "Chegada Prevista" ]).dt.seconds / 60
voos[ "atraso_saida" ] = (voos[ "Partida Real" ] - voos[ "Partida Prevista" ]).dt.seconds / 60
voos[ "atraso_ajustado" ] = voos[ "atraso_chegada" ] - voos[ "atraso_saida" ]
voos[ "atraso_chegada_abs" ] = abs( voos[ "atraso_chegada" ] )
voos[ "atraso_saida_abs" ] = abs( voos[ "atraso_saida" ] )
voos[ "atraso_ajustado_abs" ] = abs( voos[ "atraso_ajustado" ] )

In [ ]:
voos[ "atraso_chegada" ].plot( kind = "hist", bins = 100 )

In [ ]:
voos[ "duracao_real" ] = (voos[ "Chegada Real" ] - voos[ "Partida Real" ]).dt.seconds / 60
voos[ "duracao_prevista" ] = (voos[ "Chegada Prevista" ] - voos[ "Partida Prevista" ]).dt.seconds / 60

In [ ]:
(voos.query( "atraso_chegada > 0 & atraso_chegada < 240" ).groupby( [ "ano", "mes" ] )
 .agg(
		percentil_10 = ("atraso_chegada", lambda x: x.quantile( 0.10 )),
		percentil_20 = ("atraso_chegada", lambda x: x.quantile( 0.20 )),
		percentil_30 = ("atraso_chegada", lambda x: x.quantile( 0.30 )),
		percentil_40 = ("atraso_chegada", lambda x: x.quantile( 0.40 )),
		percentil_50 = ("atraso_chegada", lambda x: x.quantile( 0.50 )),
		percentil_60 = ("atraso_chegada", lambda x: x.quantile( 0.60 )),
		percentil_70 = ("atraso_chegada", lambda x: x.quantile( 0.70 )),
		percentil_80 = ("atraso_chegada", lambda x: x.quantile( 0.80 )),
		percentil_90 = ("atraso_chegada", lambda x: x.quantile( 0.90 )),
		)[ [ "percentil_10",
             "percentil_20",
             "percentil_30",
             "percentil_40",
             "percentil_50",
             "percentil_60",
             "percentil_70",
             "percentil_80",
             "percentil_90" ] ].sort_index( ascending = True ).plot( figsize = (20, 20) )
 )
# Doideira : um salto gigantesco no tamanho dos atrasos_saida e chegada, mas ambos acompanham uns aos outros.
# #Uma possivel explicação é a presença da olimpiada no mesmo ano /periodo em que vemos esse crescimento acentuado. No entanto, era de se esperar que passado esse evento, tudo se normalizasse, mas
# não é o que observamos.


In [ ]:
voos.query( "atraso_chegada > 0" ).groupby( [ "ano", "mes" ] )[ [ "atraso_chegada",
                                                                  "atraso_saida" ] ].mean().sort_index( ascending = True ).plot()

# Analisando somente os voos que atrasam, Vemos que a duração desses atrasos permanece razoavelmente constante. ao redor de 600 +- 100, Com excessão desse evento proximo a 2017-1

In [ ]:
# Sera que os atrasos seguem algum tipo de padrão sazonal?

voos.groupby( "semana_ano" ).mean()[ [ "atraso_chegada",
                                       "atraso_saida" ] ].plot()

# Temos um descolamento entre  chegada e saida ao redor da semana 40.

# Nao estou vendo muito padrão nao. Talvez necessitamos de mais variaveis, ou talvez seja com o dia do mes / dia da semana.
# Talvez durante a transição de horario de verão?

In [ ]:
voos.query( "atraso_chegada > 0 " ).groupby( "dia_semana" ).median()[ [ "atraso_chegada",
                                                                        "atraso_saida" ] ].plot()

# Parece haver um efeito sazonal do dia da semana no que diz respeito aos atrasos.
# Peak atrasos ~ (terca )
# Menor ~ ( quinta)

In [ ]:

ids_linhas = ['N' , 'E' , 'R']
ids_dis = [ '0' , '4' , 'C']

voos.rename( {
		'Código Tipo Linha' : 'id_tipo_linha',
		'Código Autorização (DI)' : 'cd_di'
		} , axis = 1)\
	.query('id_tipo_linha in @ids_linhas')\
	.query('cd_di in @ids_dis')\


# Filtrando apenas pelos municípios de SP e RJ, vemos que existem algumas empresas que nem conhecemos, bem como aeroportos que não conhecemos.

# Um filtro que faremos daqui em diante vai ser filtrar apenas voos de passageiros.
# Isso signfica apenas os códigos ( N , I, E , R).

# Um segundo filtro diz respeito aos voos internacionais: Não consideramos eles.
# Isso inviabiliza os voos com código I.

# Portanto consideramos apenas os códigos ( N , E , R )


In [ ]:
# Outra dimensão a ser considerada é o Digito Identificador.
# Ele diz respeito ao tipo de voo realizado. Podem ser dos seguintes tipos:
# Regular, Improdutivo, Não regular.
# Nesta analise consideraremos apenas os voos regulares.
# Estes são sinalizados com cd_di pertencente à ( 0 , 4 , C)

In [ ]:
# Outro ponto a ser considerado é a situação do voo.
# Consideraremos apenas os voos Realizados

In [ ]:
# Correlação de atrasos com outras features
# Algum aeroporto atrasa mais que os outros? Isso pode ser causado pelo clima do aeroporto ou pela movimentação daquele aeroporto
# Horario do dia. ( Período manhã / tarde / noite )
# Sera que algum voo atrasa mais?
# Alguma companhia aerea?

# Lotação?


voos.columns